In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')#открыть доступ к Google диску для загрузки снимков

Mounted at /content/gdrive


In [ ]:
!pip3 install git+https://github.com/qubvel/segmentation_models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-9z5tv9bg
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-9z5tv9bg
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33810 sha256=cc7f01ab8e93047b61c8b064a5ad4172fecce0097b756e3d340ad32a9c7f7b15
  Stored in directory: /tmp/pip-ephem-wheel-cache-8r9pp1oi/wheels/02/cd/18/61c0bbb8766acfec68f9d20618886b7b38dfeeb95865b6ba00
Successfully built segmentation-models


In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt

In [ ]:
import keras
import keras.backend as K
from keras import optimizers
from osgeo import gdal
import segmentation_models as sm

Segmentation Models: using `keras` framework.


In [ ]:
window_size = 64
i_bands = 3
BACKBONE = 'resnet34' 
model = sm.Unet(BACKBONE, classes=3, encoder_weights=None)
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/bestmodel.h5")#путь до предобученых весов для нейронной сети

In [11]:
!pip install fels
!fels 48UVC S2 2019-06-20 2019-07-05 -o "/content/gdrive/My Drive/Colab Notebooks/input" --outputcatalogs /tmp --noinspire 
#запрос на сайт Google cloud. Указывается участок земли: 48UVC(Ирк обл), дата начала, дата конца и путь куда сохранить архивы

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.3 MB 25.4 MB/s 
     |████████████████████████████████| 1.0 MB 71.1 MB/s 
     |████████████████████████████████| 16.7 MB 58.9 MB/s 
     |████████████████████████████████| 6.3 MB 71.1 MB/s 
Unzipping Metadata file...
Searching for Sentinel-2 images in catalog...
Found 5 files.


In [ ]:
image = []
path = "/content/gdrive/My Drive/Colab Notebooks/input"#папка в которую скачаны снимки с сайти Google cloud
for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith("TCI.jp2"):
      image.append(os.path.join(root,file))
print(image)


['/content/gdrive/My Drive/Colab Notebooks/input/S2A_MSIL1C_20190526T040551_N0207_R047_T48UVC_20190526T064949.SAFE/GRANULE/L1C_T48UVC_A020493_20190526T040647/IMG_DATA/T48UVC_20190526T040551_TCI.jp2', '/content/gdrive/My Drive/Colab Notebooks/input/S2A_MSIL1C_20190529T041551_N0207_R090_T48UVC_20190529T071832.SAFE/GRANULE/L1C_T48UVC_A020536_20190529T041552/IMG_DATA/T48UVC_20190529T041551_TCI.jp2', '/content/gdrive/My Drive/Colab Notebooks/input/S2B_MSIL1C_20190531T040549_N0207_R047_T48UVC_20190531T101646.SAFE/GRANULE/L1C_T48UVC_A011656_20190531T040650/IMG_DATA/T48UVC_20190531T040549_TCI.jp2', '/content/gdrive/My Drive/Colab Notebooks/input/S2B_MSIL1C_20190603T041559_N0207_R090_T48UVC_20190603T075608.SAFE/GRANULE/L1C_T48UVC_A011699_20190603T041720/IMG_DATA/T48UVC_20190603T041559_TCI.jp2', '/content/gdrive/My Drive/Colab Notebooks/input/S2B_MSIL1C_20190524T041559_N0207_R090_T48UVC_20190524T070158.SAFE/GRANULE/L1C_T48UVC_A011556_20190524T041622/IMG_DATA/T48UVC_20190524T041559_TCI.jp2']


In [ ]:
from osgeo import gdal
from numba import jit
from PIL import Image

In [ ]:
@jit(nopython=True)
def reverse(a,b):
  for i in range(b.shape[1]):
    for j in range(b.shape[2]):
      for x in range(b.shape[0]):
        a[i,j,x] = b[x,i,j]
  return a      

In [ ]:
image_list = []
for img in image:
  m_ds = gdal.Open(img)
  m_band = m_ds.GetRasterBand(1)
  m_arr = m_ds.ReadAsArray()
  a = np.zeros([len(m_arr[1]), len(m_arr[2]), 3], np.uint8)
  img1 = reverse(a, m_arr)
  height = 1536
  ratio = (height / float(img1.shape[1]))
  width = int((float(img1.shape[0]) * float(ratio)))
  image_resize = cv2.resize(img1, (width, height))
  image_list.append(image_resize)

In [ ]:
'''input_path = "/content/gdrive/My Drive/Colab Notebooks/input" 
#считывание снимков по заданной дириктории, нужно указать путь до них и расширение

name_mask = '*.jpg'
path = os.path.join(input_path, name_mask)
files = glob.glob(path)
image_list = []

for img in files: 
  image = cv2.imread(img)
  image_list.append(image)
print(len(image_list))'''

1


In [ ]:
imagelist = []
masklist = []
for image in image_list:
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  height, width, channels = image.shape
  x = height // window_size
  y = width // window_size
  image = image[0:x*window_size,0:y*window_size]

  height, width, channels = image.shape
  print (height, width)
  seg = np.zeros((height, width, channels), np.uint8)

  i = 0
  while i <= (height-window_size):
    j = 0
    while j <= (width-window_size):
      ROI = image[i:i+window_size, j:j+window_size]
      
      result = np.ndarray(shape=(1,window_size,window_size,i_bands), dtype='float32')
      result[0,:,:,:] = ROI    
      result = result / 255
      out = model.predict(result, verbose=0, steps=None).round() 
      ROIres = out[0,:,:,:] *255
      seg[i:i+window_size, j:j+window_size] = ROIres
      j += window_size
    i += window_size  
  
  fig, axes = plt.subplots(1, 2, figsize=(30, 30))
  axes[0].set_axis_off()
  axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  axes[1].set_axis_off()
  axes[1].imshow(cv2.cvtColor(seg, cv2.COLOR_BGR2RGB))
  imagelist.append(image)
  masklist.append(seg)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
@jit(nopython=True)
def cloud(mask):
  cloudmin = 100
  minimage = 0
  all = mask[0].shape[0]*mask[0].shape[1]
  for i in range(len(mask)):
    image = mask[i]
    cloud = 0
    for x in range(image.shape[0]):
      for y in range(image.shape[1]):
        if (image[x,y,0] != 0 or image[x,y,1] != 0 or image[x,y,2] != 0):
          cloud += 1
    if (cloud/all < cloudmin):
      cloudmin = cloud/all
      minimage = i
  return minimage    

In [ ]:
mincloud = cloud(masklist)
print(mincloud)

/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'mask' of function 'cloud'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-12-b49512fa513c>", line 2:
@jit(nopython=True)
def cloud(mask):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


2


In [ ]:
@jit
def overlay(img_mask,img_mask1, img, img1):
  for i in range (img_mask.shape[0]):
      for j in range (img_mask.shape[1]):
        if ((img_mask[i,j,0] != 0 or img_mask[i,j,1] != 0 or img_mask[i,j,2] != 0) and (img_mask1[i,j,0] == 0 and img_mask1[i,j,1] == 0 and img_mask1[i,j,2] == 0) and (img1[i,j,0] != 0 and img1[i,j,1] != 0 and img1[i,j,2] != 0)):
          img[i,j, :] = img1[i,j, :] 
          img_mask[i,j,:] = [0,0,0]
        if (img[i,j,0] == 0 and img[i,j,1] == 0 and img[i,j,2] == 0 and img1[i,j,0] != 0 and img1[i,j,1] != 0 and img1[i,j,2] != 0):
          img[i,j, :] = img1[i,j, :] 
          img_mask[i,j,:] = img_mask1[i,j,:]
  return img, img_mask

In [ ]:
img = np.array(imagelist[mincloud])
img_mask = np.array(masklist[mincloud])
original = imagelist[mincloud]
for k in range (0, len(imagelist)):
  if (k != mincloud):
    height, width, channels = img.shape
    print (height, width)

    img1 = np.array(imagelist[k])
    img_mask1 = np.array(masklist[k])
    img, img_mask = overlay(img_mask,img_mask1, img, img1)
    
  
    fig, axes = plt.subplots(1, 2, figsize=(30, 30))
    axes[0].set_axis_off()
    axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    axes[1].set_axis_off()
    axes[1].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    fig, axes = plt.subplots(1, 2, figsize=(30, 30))
    axes[0].set_axis_off()
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[1].set_axis_off()
    axes[1].imshow(cv2.cvtColor(img_mask, cv2.COLOR_BGR2RGB))
    original = img
cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/img.bmp', img)#путь и название финального снимка с меньшей облачностью

Output hidden; open in https://colab.research.google.com to view.